In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from open_pref_eval.datasets import ds2name, load_dataset_n
from open_pref_eval.datasets.genies import GENIES, dist2datasets, GENIES_ALL

from reprpo.data.collate3 import TokenizeRow
from reprpo.models.load import load_model

import gc
from loguru import logger

In [3]:
import numpy as np
import pandas as pd

In [4]:

model, tokenizer = load_model(
    "wassname/llama-3-2-1b-sft",
)




In [5]:
tokenize_row = TokenizeRow(
    tokenizer,
    max_length=2048,
    max_prompt_length=512,
)

In [6]:
df = pd.DataFrame(GENIES_ALL)
s = df['source'].tolist() + df['target'].tolist() #+ df['target_reference'].tolist()
s = list(set(s))
s

['ranking_logic_hard',
 'spanish_input',
 'us_history_textbook',
 'sycophancy_feedback',
 'code_low_quality',
 'raven_matrices',
 'unhelpful_alpaca',
 'code_is_correct',
 'alpaca_mmlu',
 'math',
 'survival_influence',
 'change_my_view',
 'comma_separated_input',
 'spanish_output',
 'personality_traits',
 'code_easy',
 'code',
 'sycophancy_answer',
 'us_history',
 'gender_bias',
 'relinquish_power',
 'wrong_arc',
 'ranking_logic',
 'creative_writing',
 'math_fiction',
 'word_swap',
 'cooking',
 'alpaca_long',
 'alpaca_short',
 'ranking_logic_easy',
 'shp_low_quality',
 'math_textbook',
 nan,
 'crt_2',
 'arc_hard',
 'crt_3',
 'biology_with_literary_style',
 'math_easy',
 'alpaca_high_quality',
 'counterfactual_python',
 'punishment_avoidance',
 'alpaca_hard',
 'sycophancy_mimicry',
 'illegal_dont_help',
 'raven_easy',
 'pursue_goals',
 'math_make_questions',
 'alpaca_chat',
 'comma_separated_output',
 'shp_high_quality',
 'us_history_make_questions',
 'arc_easy',
 'alpaca_low_quality',
 

In [8]:
# ds2 = ds.map(lambda x: {
#     'prompt_len': len(tokenizer(x['prompt'])[0]),
#     'chosen_len': len(tokenizer(x['chosen'])[0]),
#     'rejected_len': len(tokenizer(x['rejected'])[0]),
#     }
# )
# ds2

In [36]:
df = ds2.select_columns(['prompt_len', 'chosen_len', 'rejected_len']).to_pandas()
d = df.describe()
d

,prompt_len,chosen_len,rejected_len
count,100.000000,100.000000,100.000000
mean,117.050000,110.210000,110.230000
std,48.153873,35.468949,35.227645
min,69.000000,43.000000,43.000000
25%,93.000000,80.250000,80.250000
50%,103.000000,106.000000,105.000000
75%,116.000000,138.000000,138.000000
max,268.000000,170.000000,170.000000


In [40]:
# df.quantile(0.99)
# # df.max()

In [14]:
from tqdm.auto import tqdm

data = []
for dist in tqdm(s):
    try:
        ds = load_dataset_n("wassname/genies_preferences", name=dist,   split='train', N=100)
        ds2 = ds.map(lambda x: {
            'prompt_len': len(tokenizer(x['prompt'])[0]),
            'chosen_len': len(tokenizer(x['chosen'])[0]),
            'rejected_len': len(tokenizer(x['rejected'])[0]),
            }
        )
        df = ds2.select_columns(['prompt_len', 'chosen_len', 'rejected_len']).to_pandas()
        


        # ds_train_tok = ds.map(tokenize_row, batched=False)

        gc.collect()
        data.append(dict(
            dist=dist, 
            prompt_len_mean=df['prompt_len'].mean(),
            prompt_len_std=df['prompt_len'].std(),
            prompt_len_min=df['prompt_len'].min(),
            prompt_len_max=df['prompt_len'].max(),
            chosen_len_mean=df['chosen_len'].mean(),
            chosen_len_std=df['chosen_len'].std(),
            chosen_len_min=df['chosen_len'].min(),
            chosen_len_max=df['chosen_len'].max(),
            rejected_len_mean=df['rejected_len'].mean(),
            rejected_len_std=df['rejected_len'].std(),
            rejected_len_min=df['rejected_len'].min(),
            rejected_len_max=df['rejected_len'].max(),
            
                            ))
        
    except ValueError as e:
        print(dist, e)
    except Exception as e:
        print(dist, e)
        # raise e


  0%|          | 0/62 [00:00<?, ?it/s]

code_is_correct BuilderConfig 'code_is_correct' not found. Available: ['alpaca_chat', 'alpaca_easy', 'alpaca_hard', 'alpaca_high_quality', 'alpaca_long', 'alpaca_low_quality', 'alpaca_mmlu', 'alpaca_short', 'arc_easy', 'arc_hard', 'biology_with_literary_style', 'change_my_view', 'code', 'code_easy', 'code_hard', 'code_low_quality', 'comma_separated_input', 'comma_separated_output', 'cooking', 'counterfactual_python', 'creative_writing', 'crt_1', 'crt_2', 'crt_3', 'gender_bias', 'illegal_dont_help', 'math', 'math_easy', 'math_fiction', 'math_hard', 'math_make_questions', 'math_textbook', 'personality_traits', 'punishment_avoidance', 'pursue_goals', 'ranking_logic', 'ranking_logic_easy', 'ranking_logic_hard', 'raven_easy', 'raven_matrices', 'relinquish_power', 'reward_seeking', 'shp_high_quality', 'shp_low_quality', 'spanish_input', 'spanish_output', 'survival_influence', 'sycophancy_answer', 'sycophancy_are_you_sure', 'sycophancy_feedback', 'sycophancy_mimicry', 'truthful_qa', 'unhelpfu

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/600 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [33]:
df = pd.DataFrame(data)
df=df.set_index('dist').sort_values('prompt_len_max')[:: -1]
df

,prompt_len_mean,prompt_len_std,prompt_len_min,prompt_len_max,chosen_len_mean,chosen_len_std,chosen_len_min,chosen_len_max,rejected_len_mean,rejected_len_std,rejected_len_min,rejected_len_max
dist,,,,,,,,,,,,
raven_matrices,283.51,83.273250,172,494,26.70,11.836001,12,59,28.15,15.078498,12,91
us_history,125.86,121.267883,58,477,65.11,50.582645,3,165,52.43,38.353027,3,159
shp_high_quality,155.74,81.327076,63,439,112.34,87.715129,8,375,87.77,69.777548,3,371
shp_low_quality,152.70,76.163235,69,416,87.33,77.350449,4,317,55.77,49.071553,2,255
code_low_quality,157.62,90.702509,76,385,113.00,72.350159,29,336,111.85,70.536669,29,346
code,127.38,74.565920,63,380,101.52,72.211457,12,294,95.64,61.897687,9,294
change_my_view,252.11,47.784638,169,377,107.27,58.332183,12,260,78.81,52.423238,5,256
code_hard,193.34,80.733283,74,355,153.39,62.680510,81,361,141.37,56.079787,33,365
counterfactual_python,132.89,48.637163,78,302,82.03,67.057409,11,271,77.21,61.778080,14,279


In [34]:
df[['prompt_len_max', 'chosen_len_max']]

,prompt_len_max,chosen_len_max
dist,,
raven_matrices,494,59
us_history,477,165
shp_high_quality,439,375
shp_low_quality,416,317
code_low_quality,385,336
code,380,294
change_my_view,377,260
code_hard,355,361
counterfactual_python,302,271
